### *Load and preprocess the dataset*

In [4]:
# import pandas as pd
# from datetime import timedelta

# # Load the dataset CSV file into a pandas DataFrame.
# # Use encoding='ISO-8859-1' because the dataset contains special characters.

    
# df = pd.read_csv(r'Data\Online Retail.csv', encoding='ISO-8859-1')


# # Convert the 'InvoiceDate' column to datetime type for easier date manipulation.
# df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# # Shift the InvoiceDate forward by 14 years to adjust the dataset date range from 2010-2011 to approximately 2024-2025.
# df['InvoiceDate'] = df['InvoiceDate'] + pd.DateOffset(years=14)

# # Preview the earliest and latest dates after shifting to confirm the date range.
# print("Date range after shifting:", df['InvoiceDate'].min(), "to", df['InvoiceDate'].max())
